## Provided circle generator code

In [ ]:
from typing import NamedTuple, Optional, Tuple, Generator

import numpy as np
from matplotlib import pyplot as plt
from skimage.draw import circle_perimeter_aa


class CircleParams(NamedTuple):
    row: int
    col: int
    radius: int


def draw_circle(img: np.ndarray, row: int, col: int, radius: int) -> np.ndarray:
    """
    Draw a circle in a numpy array, inplace.
    The center of the circle is at (row, col) and the radius is given by radius.
    The array is assumed to be square.
    Any pixels outside the array are ignored.
    Circle is white (1) on black (0) background, and is anti-aliased.
    """
    rr, cc, val = circle_perimeter_aa(row, col, radius)
    valid = (rr >= 0) & (rr < img.shape[0]) & (cc >= 0) & (cc < img.shape[1])
    img[rr[valid], cc[valid]] = val[valid]
    return img


def noisy_circle(
        img_size: int, min_radius: float, max_radius: float, noise_level: float
) -> Tuple[np.ndarray, CircleParams]:
    """
    Draw a circle in a numpy array, with normal noise.
    """

    # Create an empty image
    img = np.zeros((img_size, img_size))

    radius = np.random.randint(min_radius, max_radius)

    # x,y coordinates of the center of the circle
    row, col = np.random.randint(img_size, size=2)

    # Draw the circle inplace
    draw_circle(img, row, col, radius)

    added_noise = np.random.normal(0.5, noise_level, img.shape)
    img += added_noise

    return img, CircleParams(row, col, radius)


def show_circle(img: np.ndarray):
    fig, ax = plt.subplots()
    ax.imshow(img, cmap='gray')
    ax.set_title('Circle')
    plt.show()


def generate_examples(
        noise_level: float = 0.5,
        img_size: int = 100,
        min_radius: Optional[int] = None,
        max_radius: Optional[int] = None,
        dataset_path: str = 'ds',
) -> Generator[Tuple[np.ndarray, CircleParams], None, None]:
    if not min_radius:
        min_radius = img_size // 10
    if not max_radius:
        max_radius = img_size // 2
    assert max_radius > min_radius, "max_radius must be greater than min_radius"
    assert img_size > max_radius, "size should be greater than max_radius"
    assert noise_level >= 0, "noise should be non-negative"

    params = f"{noise_level=}, {img_size=}, {min_radius=}, {max_radius=}, {dataset_path=}"
    print(f"Using parameters: {params}")
    while True:
        img, params = noisy_circle(
            img_size=img_size, min_radius=min_radius, max_radius=max_radius, noise_level=noise_level
        )
        yield img, params


def iou(a: CircleParams, b: CircleParams) -> float:
    """Calculate the intersection over union of two circles"""
    r1, r2 = a.radius, b.radius
    d = np.linalg.norm(np.array([a.row, a.col]) - np.array([b.row, b.col]))
    if d > r1 + r2:
        # If the distance between the centers is greater than the sum of the radii, then the circles don't intersect
        return 0.0
    if d <= abs(r1 - r2):
        # If the distance between the centers is less than the absolute difference of the radii, then one circle is
        # inside the other
        larger_r, smaller_r = max(r1, r2), min(r1, r2)
        return smaller_r ** 2 / larger_r ** 2
    r1_sq, r2_sq = r1**2, r2**2
    d1 = (r1_sq - r2_sq + d**2) / (2 * d)
    d2 = d - d1
    sector_area1 = r1_sq * np.arccos(d1 / r1)
    triangle_area1 = d1 * np.sqrt(r1_sq - d1**2)
    sector_area2 = r2_sq * np.arccos(d2 / r2)
    triangle_area2 = d2 * np.sqrt(r2_sq - d2**2)
    intersection = sector_area1 + sector_area2 - (triangle_area1 + triangle_area2)
    union = np.pi * (r1_sq + r2_sq) - intersection
    return intersection / union


if __name__ == '__main__':
    img, params = noisy_circle(100, 10, 20, 0.2)
    show_circle(img)
    print(params)
    print(iou(params, CircleParams(50, 50, 10)))

## Define circle hyperparameters

In [3]:
IMG_SIZE = 100 # Default is 100
NOISE_LEVEL = 0.5 # Default is 0.5
MIN_RAD = IMG_SIZE // 10
MAX_RAD = IMG_SIZE // 2

## Define CNN using PyTorch

In [ ]:
import torch

print(torch.__version__)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CircleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 5), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2, 2)
        )   # 1 input channel, since it's greyscale
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 3), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2, 2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, 3), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2, 2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, 3), nn.BatchNorm2d(256), nn.ReLU()
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(256, 512, 1), nn.BatchNorm2d(512), nn.ReLU()
        )

        # Calculate the output size after the last convolutional layer
        self.fc_input_size = self._calculate_conv_output_size((1, IMG_SIZE, IMG_SIZE))

        self.fc1 = nn.Linear(self.fc_input_size, 256)  # Adjusted input size
        self.fc2 = nn.Linear(256, 32)
        self.fc3 = nn.Linear(32, 3)   # 3 output units for (x, y, radius)

    def _calculate_conv_output_size(self, input_size):
        with torch.no_grad():
            input_tensor = torch.ones(1, *input_size)
            output_tensor = self._forward_conv(input_tensor)
            # Return flattened to correspond to fc layer input
            return output_tensor.view(1, -1).size(1)

    def _forward_conv(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        return x

    def forward(self, x):
        x = self._forward_conv(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))  # Apply ReLU activation
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Output (x, y, radius) predictions
        return x


model = CircleNet()

# Calculate the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters:", total_params)


## Define dataset

In [6]:
from torch.utils.data import Dataset
import random


class CircleDataset(Dataset):
    def __init__(
        self,
        num_samples,
        noise_level=NOISE_LEVEL,
        img_size=IMG_SIZE,
        min_radius=None,
        max_radius=None,
    ):
        self.num_samples = num_samples
        if noise_level == -1:
            noise_level = random.uniform(
                0, 0.7
            )  # Randomly choose noise level between 0 and 0.7
        self.generator = generate_examples(
            noise_level=noise_level,
            img_size=img_size,
            min_radius=min_radius,
            max_radius=max_radius,
        )

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        img, params = next(self.generator)
        # Preprocess image
        img = torch.Tensor(img).unsqueeze(0)  # Since input channels = 1
        # We want to predict the circle params
        label = torch.Tensor(params)
        return img, label


## Train model

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader

# Define hyperparameters
num_epochs = 10
batch_size = 32
num_samples = 120000  # Number of samples in the dataset

# Create dataset and DataLoader
dataset = CircleDataset(num_samples=num_samples, noise_level=NOISE_LEVEL)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = nn.DataParallel(model)
model.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 0:  # Print every 200 mini-batches
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 200}")
            running_loss = 0.0

print("Finished Training")


## Save model

In [17]:
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive")

# Define path to save model
model_save_path = "/content/drive/My Drive/circle_challenge/circle_model.pth"


# Save model weights
torch.save(model.state_dict(), model_save_path)

Mounted at /content/drive


## Evaluate model

In [ ]:
# List of different amounts of noise
noise_levels = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

# Evaluate model for each noise level
avg_iou_per_noise = {}
for noise_level in noise_levels:
    # Generate test dataset with the current noise level
    num_test_samples = 100
    test_dataset = [
        noisy_circle(IMG_SIZE, MIN_RAD, MAX_RAD, noise_level)
        for _ in range(num_test_samples)
    ]

    # Evaluate model
    model.eval()  # Set model to evaluation mode
    iou_scores = []
    for img, params_gt in test_dataset:
        # Convert image to tensor and add batch dimension
        img_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

        # Predict circle parameters
        with torch.no_grad():
            params_pred = model(img_tensor)

        # Convert predicted parameters to CircleParams format
        params_pred = CircleParams(*params_pred.squeeze().tolist())

        # Calculate IoU
        iou_score = iou(params_gt, params_pred)
        iou_scores.append(iou_score)

    # Calculate average IoU for the current noise level
    avg_iou = sum(iou_scores) / len(iou_scores)
    avg_iou_per_noise[noise_level] = avg_iou

# Print average IoU for each noise level
for noise_level, avg_iou in avg_iou_per_noise.items():
    print(f"Average IoU for noise level {noise_level}: {avg_iou}")
